# Ejercicio de introducción a Pytorch
Haremos un recorrido por los aspectos fundamentales de pytroch desde el manejo de tensores hasta el entrenamiento y evaluación de una red neuronal.
Para completarlo llevamos como guía [ESTE](https://colab.research.google.com/github/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial2/Introduction_to_PyTorch.ipynb#scrollTo=u-L7YQmcHvX8) cuaderno.y otros recursos dados a lo largo del cuaderno.


Primero importamos algunas librerías básicas

In [2]:
## Standard libraries
import os
import math
import numpy as np
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgba
import seaborn as sns
sns.set()

## Progress bar
from tqdm.notebook import tqdm


<ipython-input-2-ef889392170e>:11: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


In [3]:
#Pytorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torch.utils.data
import torchvision.transforms as transform

Primero recordemos algunas funconalidades de los tensores.

In [4]:
# Crear un tensor aleatorio con entradas entre 0 y 1, de tamaño 3x3
primer_tensor = torch.rand(3, 3)

# Crear un tensor de tamaño 3x3 con valores en una distribución normal estandar
segundo_tensor = torch.randn(3, 3)

# Calcular el tamaño de los tensores
tensor_size = primer_tensor.size()

# Imprimir los valores de los vectores y su tamaño

print("Primer Tensor (Aleatorio 3x3):")
print(primer_tensor)
print("Tamaño del Primer Tensor:", tensor_size)

print("\nSegundo Tensor (Normal Estándar 3x3):")
print(segundo_tensor)
print("Tamaño del Segundo Tensor:", segundo_tensor.size())


Primer Tensor (Aleatorio 3x3):
tensor([[0.3790, 0.8204, 0.9024],
        [0.8088, 0.1526, 0.2901],
        [0.9436, 0.6298, 0.7912]])
Tamaño del Primer Tensor: torch.Size([3, 3])

Segundo Tensor (Normal Estándar 3x3):
tensor([[ 1.4567, -1.5909, -1.4519],
        [ 1.3838,  1.4801, -0.4147],
        [-1.1376, -1.0288,  1.0429]])
Tamaño del Segundo Tensor: torch.Size([3, 3])


In [6]:
# Crear una matriz de unos de tamaño 3 by 3
tensor_of_ones = torch.ones(3, 3)

# Crear una matrix identidad de tamaño 3 by 3
identity_tensor = torch.eye(3)

# Multiplicar las dos matrices anteriores
matrices_multiplied = torch.matmul(tensor_of_ones, identity_tensor)
print(matrices_multiplied)

# ¿Qué ocurre si las multiplica usando * ?

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])


En cuanto a la multiplicación con el operador *, esta realizará una multiplicación elemento a elemento (multiplicación Hadamard) en lugar de una multiplicación de matrices. En este caso, ambas matrices tienen el mismo tamaño, por lo que la multiplicación con * funcionaría correctamente y multiplicaría cada elemento correspondiente de ambas matrices. Sin embargo, para realizar una multiplicación matricial adecuada, debes utilizar la función torch.matmul como se muestra en el ejemplo anterior.

### Cálculo de gradientes
Calculemos un gradiente utilizando Pytorch. La función es la siguiente:

<center style="width: 100%"><img src="https://drive.google.com/uc?export=view&id=1rAmD2ZzVGm6bPj7DVYyhQ1tYZPTQFqAi" width="700px"></center>

Para esto, puede ir a la sección Dynamic Computation Graph and Backpropagation, del cuaderno inicial.

In [9]:
# Initialize x, y and z to values 4, -3 and 5
x = torch.tensor(4.0, requires_grad=True)
y = torch.tensor(-3.0, requires_grad=True)
z = torch.tensor(5.0, requires_grad=True)

# Set q to sum of x and y, set f to product of q with z
q = x + y
f = q * z

# Compute the derivatives
f.backward()

# Print the gradients
print("Gradient of x is: " + str(x.grad))
print("Gradient of y is: " + str(y.grad))
print("Gradient of z is: " + str(z.grad))

Gradient of x is: tensor(5.)
Gradient of y is: tensor(5.)
Gradient of z is: tensor(1.)


Ahora calculemos los gradientes para la función descrita en la siguiente imagen:

<center style="width: 100%"><img src="https://drive.google.com/uc?export=view&id=1WaHCg-h4nz7PTGwYlM5R9EX8n_WNv3jI" width="700px"></center>

In [10]:
# Initializar x,y,z como tensores aleatorios de tamaño (1000,1000)
x = torch.randn(1000, 1000, requires_grad=True)
y = torch.randn(1000, 1000, requires_grad=True)
z = torch.randn(1000, 1000, requires_grad=True)

# Multiplicar los tensores x con y
q = x * y

# Multiplicar componente a componente los tensores z con q
f = z * q

mean_f = torch.mean(f)

# Calcular los gradientes
mean_f.backward()

# Print the gradients
print("Gradient of x is: " + str(x.grad))
print("Gradient of y is: " + str(y.grad))
print("Gradient of z is: " + str(z.grad))

Gradient of x is: tensor([[-3.2087e-07,  6.9688e-07,  4.8367e-07,  ...,  1.7413e-07,
          2.5891e-07, -1.3880e-06],
        [ 7.5873e-07, -1.6251e-06,  8.2395e-07,  ...,  1.8550e-06,
         -6.4307e-07,  2.2686e-06],
        [-3.6292e-07, -1.7868e-06,  6.8491e-07,  ..., -8.3328e-07,
          4.2020e-08, -7.3244e-07],
        ...,
        [-5.2999e-07, -2.5889e-06, -1.5398e-06,  ...,  2.1023e-06,
         -2.5166e-07,  1.7977e-07],
        [ 2.3449e-07,  1.6045e-06,  3.2788e-08,  ...,  3.4410e-08,
          4.2217e-07, -8.7365e-09],
        [ 8.4237e-07,  1.6198e-08, -1.3614e-06,  ...,  1.5465e-07,
         -1.2403e-07, -3.0991e-08]])
Gradient of y is: tensor([[ 1.8797e-09,  2.0787e-07, -4.5042e-07,  ...,  1.7788e-07,
          7.6858e-07, -9.6075e-07],
        [-7.1322e-07,  1.1403e-06,  3.6076e-07,  ...,  1.1040e-06,
         -7.7504e-07, -3.4933e-06],
        [-9.0009e-08,  7.8000e-07,  1.0777e-06,  ..., -4.2923e-07,
          3.6508e-08, -2.0577e-06],
        ...,
        [-

### Construcción de redes neuronales con Pytorch

Construimos una red neuronal en Pytorch de forma *manual*. la entrada serán imágenes de tamaño (28,28). Es decir contienen pixeles de 784 pixeles.
La red contendrá una capa de entrada, una capa oculta con 200 unidades y una capa de salida con 10 categorías.

In [18]:
input_layer=torch.rand(784)
# Inicializar los pesos de la red neuronal
weight_1 = torch.rand(784, 200)  # Peso de la capa de entrada a la capa oculta
weight_2 = torch.rand(200, 10)  # Peso de la capa oculta a la capa de salida

# Multiplicar la capa de entrada con el peso 1
hidden_1 = torch.matmul(input_layer, weight_1)

# Multiplicar la capa oculta con el peso 2
output_layer = torch.matmul(hidden_1, weight_2)
print(output_layer)

tensor([19004.0312, 19508.8262, 18846.4883, 17985.2012, 18910.8184, 18079.0840,
        20345.4258, 19242.9590, 18945.8887, 19821.1016])


Ahora construimos la misma rede neuronal pero utilizando los módulos de Pytorch. (Ver sección *The model* del cuaderno)

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # Inicializar las dos capas lineales
        self.fc1 = nn.Linear(784, 200)
        self.fc2 = nn.Linear(200, 10)

    def forward(self, x):

        # Usar las capas inicializadas y devolver x
        x = self.fc1(x)
        x = self.fc2(x)
        return x

Construyamos la red neuronal en la gráfica siguiente, de forma *manual*


<center style="width: 100%"><img src="https://drive.google.com/uc?export=view&id=1UiW6bdXwe_jnAFU29-7P4Du1S4SqJa3R" width="700px"></center>



In [15]:
# Crear tensor aleatorio como capa de entrada
input_layer= torch.rand(784)

# Crear matrices de pesos
weight_1= torch.rand(784, 200)
weight_2= torch.rand(200, 200)
weight_3= torch.rand(200, 10)

# Calcular la primera y segunda capa oculta

hidden_1 = torch.matmul(input_layer, weight_1)
hidden_2 = torch.matmul(hidden_1, weight_2)

# Imprimir la salida
print(torch.matmul(hidden_2, weight_3))


tensor([2066741.3750, 1862685.5000, 1673508.6250, 2083226.6250, 1964582.6250,
        1918253.7500, 1945435.2500, 2014283.5000, 2008317.2500, 2120144.0000])


La anterior era una red neuronal con 2 capas ocultas ocultas en donde no se aplica ninguna función no-lineal. Veamos que ésta se puede construir con una sola capa oculta.

In [16]:
# Calcular la compuesta de las matrices de pesos
weight_composed_1 = torch.matmul(weight_1, weight_2)
weight = torch.matmul(weight_composed_1, weight_3)

# Multiplicar la capa de entrada por weight e imprimir
print(torch.matmul(input_layer, weight))

tensor([2066742.2500, 1862686.1250, 1673507.6250, 2083227.5000, 1964583.8750,
        1918253.2500, 1945434.0000, 2014284.7500, 2008317.2500, 2120143.5000])


## Entrenamiendo de una red neuronal para reconocimiento de dígitos (MNIST Dataset)
### Preparar los datos

Para preparar los datos primero creamos un parámetro *transform* para transformarlos. Haremos dos cosas:
- Transformar las imágenes del MNIST Dataset a tensores para poder alimentar la red neuronal. Esto lo hacemos con el método ToTensor.
- Por otro lado, debemos normalizarlos con respecto a una media y variaza. Esto lo hacemos con el método Normalize. En este caso usaremos una media de 0.1307 y varianza de 0.3081. (Tenga en cuenta que en el MNIST Dataset los pixeles son en escala de grises, por lo cual sólo tienen un canal de código de color.)

Para componer ambas transformaciones (Convertir a tensor y normalizar) usamos transforms.Compose ver [AQUÍ](https://www.programcreek.com/python/example/104832/torchvision.transforms.Compose)


In [20]:
import torchvision.transforms as transforms

# Definir la media y la varianza para la normalización
media = 0.1307
varianza = 0.3081

# Definir la transformación que convierte las imágenes en tensores y las normaliza
transform = transforms.Compose([
    transforms.ToTensor(),  # Convierte las imágenes en tensores
    transforms.Normalize((media,), (varianza,))  # Normaliza los tensores con la media y la varianza especificadas
])

Ahora definimos el conjunto de entrenamiento y testeo. Torchvision permite cargar datasets conocidos para visión como el MNIST.
Para entender y completar los parámetros ver [AQUÍ](https://pytorch.org/vision/main/generated/torchvision.datasets.MNIST.html).

In [21]:
# Preparar el conjunto de entrenamiento y prueba
trainset = torchvision.datasets.MNIST(root='mnist', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='mnist', train=False, download=True, transform=transform)


100%|██████████| 9912422/9912422 [00:00<00:00, 162694922.38it/s]


Extracting mnist/MNIST/raw/train-images-idx3-ubyte.gz to mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 24940435.21it/s]


Extracting mnist/MNIST/raw/train-labels-idx1-ubyte.gz to mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 50462542.11it/s]

Extracting mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 17989167.86it/s]


Extracting mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist/MNIST/raw



El método DataLoader hace parte de torch.utils.data y permite cargar los datos por lotes de un tamaño definido. Para entender los parámetros ver [AQUÍ](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader).
Preparar los datos para entrenamiento y testeo de manera que se procesen 32 imágenes cada vez y se barajen cada vez.

In [22]:
# Preparar training loader y testing loader.
# Usar los parámetros dataset, batch_size, shuffle y num_workers.
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True, num_workers=0)

Construya una clase para una red neuronal que será usada para entrenar el MNIST dataset. El dataset contiene imagenes de dimensiones (28,28,2), así que usted deducirá el tamaño de la capa de entrada. Para las calas ocultas use 200 unidades y para la capa de salida 10 unidades (una por cada categoría (Dígitos del 0 al 9)).
Como función de activación use Relu de manera funcional (nn.Functional ya está importado como F).


In [23]:
# Define the class Net
class Net(nn.Module):
    def __init__(self):
    	# Define all the parameters of the net
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 200)
        self.fc2 = nn.Linear(200, 10)

    def forward(self, x):
    	# Do the forward pass
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

###Entrenamiento del modelo

Por favor analice cuidadosamente el siguiente código, hasta que quede claro los pasos de entrenamiento y evaluación del modelo.

En primer lugar, revisemos si estamos trabajando en GPU. De lo contrario debemos cambiar el tipo de entorno de ejecución en el menú de Colab.

In [24]:
gpu_avail = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_avail}")

Is the GPU available? False


Le daremos nombre a nuestro dispositivo GPU, al cual debemos transferir nuesto modelo y los datos a utilizar.

In [25]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Device cpu


Definimos nuestro modelo

In [26]:
model=Net()
print(model)

Net(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=10, bias=True)
)


Empujamos nuestro modelo al dispositivo GPU

In [27]:
# Push model to device. Has to be only done once
model.to(device)

Net(
  (fc1): Linear(in_features=784, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=10, bias=True)
)

Definimos el ptimizador y la función de costo

In [28]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001) # descenso de gradiente
loss_module = nn.CrossEntropyLoss()  #función de costo

Entrenamos el modelo, siguiendo los 5 pasos vistos en clase

In [29]:
def train_model(model, optimizer, testloader, loss_module, num_epochs=1):
    # Set model to train mode
    model.train()

    # Training loop
    for epoch in tqdm(range(num_epochs)):
        for data_inputs, data_labels in testloader:
            data_inputs = data_inputs.view(-1, 28 * 28)
            ## Step 1: Move input data to device (only strictly necessary if we use GPU)
            data_inputs = data_inputs.to(device)
            data_labels = data_labels.to(device)



            ## Step 2: Run the model on the input data
            preds = model(data_inputs)
            preds = preds.squeeze(dim=1) # Output is [Batch size, 1], but we want [Batch size]

            ## Step 3: Calculate the loss
            loss = loss_module(preds, data_labels)

            ## Step 4: Perform backpropagation
            # Before calculating the gradients, we need to ensure that they are all zero.
            # The gradients would not be overwritten, but actually added to the existing ones.
            optimizer.zero_grad()
            # Perform backpropagation
            loss.backward()

            ## Step 5: Update the parameters
            optimizer.step()

In [30]:
train_model(model, optimizer, trainloader, loss_module)

  0%|          | 0/1 [00:00<?, ?it/s]

A continuación evaluaremos el desempeño del modelo

In [31]:
model.eval()
total, correct =0,0
for i, data in enumerate(testloader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    # Put each image into a vector
    inputs = inputs.view(-1, 784)

    # Do the forward pass and get the predictions
    outputs = model(inputs)

    _, outputs = torch.max(outputs.data, 1) #mayor valor entre los dígitos.
    total += labels.size(0)
    correct += (outputs == labels).sum().item()
print('The testing set accuracy of the network is: %d %%' % (100 * correct / total))

The testing set accuracy of the network is: 86 %


En términos de entrenamiento de redes neuronales, se adquirió conocimiento sobre el uso del descenso de gradiente estocástico (SGD) y la retropropagación para ajustar los pesos del modelo. Esto incluyó la preparación de datos, la propagación hacia adelante y hacia atrás, así como la actualización de los pesos.

También se pudo explorar la disponibilidad de GPU y cómo transferir modelos y datos entre la CPU y la GPU para acelerar el procesamiento.

En términos de rendimiento, el modelo entrenado alcanzó una precisión del 86% en el conjunto de prueba del conjunto de datos MNIST. Esto significa que el modelo es capaz de reconocer correctamente el 86% de los dígitos escritos a mano en el conjunto de prueba. En otras palabras, de cada 100 dígitos en el conjunto de prueba, el modelo clasifica correctamente 86 de ellos.

Esta precisión del 86% es un buen resultado para un modelo inicial sin ajuste de hiperparámetros ni técnicas avanzadas de entrenamiento. Sin embargo, hay margen de mejora, y mediante la optimización de hiperparámetros, arquitectura de red, aumento de datos y otras técnicas avanzadas, es posible lograr una mayor precisión en tareas de reconocimiento de dígitos.